In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Any
from math import log

In [3]:
df = pd.read_csv('queries_base_result.csv')

In [4]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [5]:
train = df[0:1606]
test = df[1607:]

In [6]:
document_response = {}
for index, row in train.iterrows():
    document_response[index] = row['Номер связки\n']

# Для natasha

In [7]:
corpus = train['Natasha']
answers = test['Natasha']

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus.values.astype(str)).toarray()

In [9]:
def vector_tfidf(query: str, vectorizer: Any) -> Any:
    new_query = []
    for word in query.split():
        if word in vectorizer.vocabulary_.keys():
            new_query.append(word)
    if len(new_query) == 0:
        new_query.append(' ')
    return vectorizer.transform(new_query).toarray()

In [10]:
def counting_top(res):
    d = {}
    # создаем словарь, где ключ - индекс документа, значение - близость документа с запросом
    index = 0
    for doc in res:
        d[index] = doc
        index += 1
    sorted_by_value = sorted(d.items(), key=lambda x: x[1], reverse=True)

    return sorted_by_value[0][0]

In [11]:
best_docs = []
for ans in answers:
    vector_query = vector_tfidf(ans, vectorizer)
    res = np.dot(X, vector_query[0])
    res = res.tolist()
    best_docs.append(counting_top(res))

In [12]:
test['natasha_tfidf'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
from rank_bm25 import BM25Okapi

In [17]:
bm25 = BM25Okapi(corpus)

In [18]:
best_docs = []
for ans in answers:
    best_doc = bm25.get_top_n(ans, corpus, n=1)[0]
    best_docs.append(corpus.tolist().index(best_doc))

In [19]:
test['natasha_bm25'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Для deepmipt

In [20]:
corpus = train['Deepmipt']
answers = test['Deepmipt']

In [21]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus.values.astype(str)).toarray()

In [22]:
best_docs = []
for ans in answers:
    vector_query = vector_tfidf(ans, vectorizer)
    res = np.dot(X, vector_query[0])
    res = res.tolist()
    best_docs.append(counting_top(res))

In [23]:
test['deepmipt_tfidf'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
bm25 = BM25Okapi(corpus)

In [25]:
best_docs = []
for ans in answers:
    best_doc = bm25.get_top_n(ans, corpus, n=1)[0]
    best_docs.append(corpus.tolist().index(best_doc))

In [26]:
test['deepmipt_bm25'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Без удаления именованных сущностей

In [28]:
corpus = train['Без удаления NE']
answers = test['Без удаления NE']

In [29]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus.values.astype(str)).toarray()

In [30]:
best_docs = []
for ans in answers:
    vector_query = vector_tfidf(ans, vectorizer)
    res = np.dot(X, vector_query[0])
    res = res.tolist()
    best_docs.append(counting_top(res))

In [31]:
test['nopreproc'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
bm25 = BM25Okapi(corpus)

In [33]:
best_docs = []
for ans in answers:
    best_doc = bm25.get_top_n(ans, corpus, n=1)[0]
    best_docs.append(corpus.tolist().index(best_doc))

In [34]:
test['nopreproc_bm25'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Для natasha + новое правило

In [35]:
corpus = train['Natasha + new rule']
answers = test['Natasha + new rule']

In [36]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus.values.astype(str)).toarray()

In [37]:
best_docs = []
for ans in answers:
    vector_query = vector_tfidf(ans, vectorizer)
    res = np.dot(X, vector_query[0])
    res = res.tolist()
    best_docs.append(counting_top(res))

In [38]:
test['natasha_date_tfidf'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
bm25 = BM25Okapi(corpus)

In [40]:
best_docs = []
for ans in answers:
    best_doc = bm25.get_top_n(ans, corpus, n=1)[0]
    best_docs.append(corpus.tolist().index(best_doc))

In [41]:
test['natasha_date_bm25'] = [document_response[n] for n in best_docs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
test.head()

,Unnamed: 0,Unnamed: 0.1,Текст вопроса,Номер связки\n,Тематика,Без удаления NE,Natasha,Deepmipt,Natasha + new rule,natasha_tfidf,natasha_bm25,deepmipt_tfidf,deepmipt_bm25,nopreproc,nopreproc_bm25,natasha_date_tfidf,natasha_date_bm25
1613,1613,1617,Здравствуйте. меня зовут Елизавета Николаевна ...,257.0,"ОКАЗАНИЕ МЕДИЦИНСКОЙ ПОМОЩИ, ГОТОВНОСТЬ СТАЦИО...",здравствовать звать елизавета николай поздняк ...,здравствовать звать 28051987гра август сообщит...,здравствовать звать 1987г р август сообщить по...,здравствовать звать 28051987гра сообщить полож...,6.0,6.0,6.0,308.0,6.0,6.0,6.0,6.0
1614,1614,1618,"То есть, если я пересекаю границу на машине, в...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",пересекать граница машина весь равно нужно сда...,пересекать граница машина весь равно нужно сда...,пересекать граница машина весь равно нужно сда...,пересекать граница машина весь равно нужно сда...,308.0,308.0,308.0,135.0,308.0,308.0,308.0,308.0
1615,1615,1619,"Добрый день. Подскажите, пожалуйста, нужно ли ...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",добрый день подсказать пожалуйста нужно турист...,добрый день подсказать пожалуйста нужно турист...,добрый день подсказать пожалуйста нужно турист...,добрый день подсказать пожалуйста нужно турист...,6.0,308.0,6.0,257.0,37.0,308.0,6.0,308.0
1616,1616,1620,Добрый день!\nМеня зовут Азиза. Я гражданка Ка...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",добрый день звать азиз гражданка казахстан про...,добрый день звать гражданка проживать супруггр...,добрый день звать гражданка проживать супруг г...,добрый день звать гражданка проживать супруггр...,6.0,308.0,6.0,6.0,37.0,308.0,6.0,132.0
1617,1617,1621,"Добрый день! Мы подрядная организация, занимае...",37.0,"ОГРАНИЧЕНИЯ, ПРОПУСКНАЯ СИСТЕМА И ПЕРЕМЕЩЕНИЕ ...",добрый день подрядный организация заниматься к...,добрый день подрядный организация заниматься к...,добрый день подрядный организация заниматься к...,добрый день подрядный организация заниматься к...,6.0,308.0,6.0,6.0,37.0,308.0,6.0,308.0


# Accuracy

In [43]:
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score

In [45]:
acc_natasha_tfidf = accuracy_score(test['Номер связки\n'], test['natasha_tfidf'])
acc_natasha_bm25 = accuracy_score(test['Номер связки\n'], test['natasha_bm25'])
acc_deepmipt_tfidf = accuracy_score(test['Номер связки\n'], test['deepmipt_tfidf'])
acc_deepmipt_bm25 = accuracy_score(test['Номер связки\n'], test['deepmipt_bm25'])
acc_nopreproc_tfidf = accuracy_score(test['Номер связки\n'], test['nopreproc'])
acc_nopreproc_bm25 = accuracy_score(test['Номер связки\n'], test['nopreproc_bm25'])
acc_natasha_date_tfidf = accuracy_score(test['Номер связки\n'], test['natasha_date_tfidf'])
acc_natasha_date_bm25 = accuracy_score(test['Номер связки\n'], test['natasha_date_bm25'])

x = PrettyTable()
x.field_names = [' ', 'natasha', 'deepmipt', 'no preprocissing', 'natasha + rules for dates']
x.add_row(['tfidf', acc_natasha_tfidf, acc_deepmipt_tfidf, acc_nopreproc_tfidf, acc_natasha_date_tfidf])
x.add_row(['bm25', acc_natasha_bm25, acc_deepmipt_bm25, acc_nopreproc_bm25, acc_natasha_date_bm25])
print(x)

+-------+--------------------+---------------------+---------------------+---------------------------+
|       |      natasha       |       deepmipt      |   no preprocissing  | natasha + rules for dates |
+-------+--------------------+---------------------+---------------------+---------------------------+
| tfidf | 0.1487481590574374 | 0.14580265095729014 | 0.12371134020618557 |    0.14727540500736377    |
|  bm25 | 0.2812960235640648 | 0.23711340206185566 |  0.2533136966126657 |    0.27393225331369664    |
+-------+--------------------+---------------------+---------------------+---------------------------+
